In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import logging

In [22]:
# LOAD DATA
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# нормализация и приведение к 32-байтовому float
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
# приведение к соответствующему типу и one-hot кодирование
y_train = keras.utils.to_categorical(y_train.astype(np.int32), 10 )
y_test = keras.utils.to_categorical(y_test.astype(np.int32) , 10)

In [25]:
def train(gan, dataset, n_epochs=5, batch_size=32, codings_size=30):
    generator, discriminator = gan.layers
    
    for ep in range(n_epochs):
        
        print("epoch #{} / {}".format(ep+1, n_epochs))
    
        for X_batch in dataset:
            # train Discriminator
            discriminator.trainable = True
            generator.trainable = False
            # генерируем нормальный шум размерности (batch size, codings size)
            noise = tf.random.normal(shape=[batch_size, codings_size])
            # получаем сгенерированные "картинки", представляющие собой просто шум
            generated = generator(noise)
            # чтобы помочь генератору научиться воссоздавать нужные нам объекты,
            # конкатенируем шумовое изображение и реальное
            X_fake_and_real = tf.concat([generated, X_batch], axis=0)
            # ?
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            # обучаем дискриминатор именно на этом пакете объектов (batch)
            discriminator.train_on_batch(X_fake_and_real, y1)
            print("train?")
            
            # train Generator
            generator.trainable = True
            discriminator.trainable = False
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)

            gan.train_on_batch(noise, y2)
            
        plot_multiple_images(generated, 6)
        plt.show()
        
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              # not shown in the book
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        plot_multiple_images(generated_images, 8)                     # not shown
        plt.show()  


In [26]:
# Определим гиперпараметры заранее
codings_size = 100
n_epochs = 10
batch_size = 32


In [27]:
# Определим сверточный генератор
dcG = keras.models.Sequential([
    keras.layers.Dense(7*7*128, input_shape=[codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, 
                                 kernel_size=5, 
                                 strides=2, 
                                 padding='SAME',
                                 activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, 
                                 kernel_size=5, 
                                 strides=2, 
                                 padding='SAME', 
                                 activation='tanh')
])

In [28]:
# Определим сверточный дискриминатор
dcD = keras.models.Sequential([
    keras.layers.Conv2D(64,
                        kernel_size=5,
                        strides=2,
                        padding='SAME',
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[28, 28,1]),
    keras.layers.Dropout(rate=0.4),
    keras.layers.Conv2D(128, 
                        kernel_size=5,
                        strides=2,
                        padding='SAME',
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [29]:

# Объединим обе последовательные модели в единую модель (обучать будем именно её)
dcgan = keras.models.Sequential([dcG, dcD])

In [30]:
# поскольку дискриминатор работает как бинарный классификатор, то соберем его именно так
dcD.compile(loss='binary_crossentropy', 
            optimizer=keras.optimizers.Adam(learning_rate=1e-2))
# аналогично цельная модель (GAN) есть бинарный классификатор
dcgan.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.Adam(learning_rate=1e-2))

In [31]:
X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1. # reshape and rescale

In [32]:

dataset_dcgan = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
dataset_dcgan = dataset_dcgan.shuffle(1000)
dataset_dcgan = dataset_dcgan.batch(batch_size, drop_remainder=True).prefetch(1)

In [34]:
train_gan(dcgan, dataset=dataset_dcgan, batch_size=32, codings_size=100)

Epoch 1/50


KeyboardInterrupt: 